<a href="https://colab.research.google.com/github/SG60/2021-multi-group-piece/blob/main/Tempos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Setup

In [3]:
from IPython.display import HTML, display


def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )


get_ipython().events.register("pre_run_cell", set_css)

from sympy import init_session

init_session()

IPython console for SymPy 1.8 (Python 3.9.3-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.8/



In [4]:
# Tool for custom rounding
from bisect import bisect_left


class CustomRound:
    def __init__(self, iterable):
        self.data = sorted(iterable)

    def __call__(self, x):
        data = self.data
        ndata = len(data)
        idx = bisect_left(data, x)
        if idx <= 0:
            return data[0]
        elif idx >= ndata:
            return data[ndata - 1]
        x0 = data[idx - 1]
        x1 = data[idx]
        if abs(x - x0) < abs(x - x1):
            return x0
        return x1


custom_round_values_base = [
    0,
    1 / 5,
    1 / 4,
    1 / 3,
    2 / 5,
    1 / 2,
    3 / 5,
    2 / 3,
    3 / 4,
    4 / 5,
]
custom_round_values = [i + j for i in range(20) for j in custom_round_values_base]
custom_round = CustomRound(custom_round_values)

# assigments

In [5]:
a, a1, b, b1, c, c1, d, s = symbols("a:2 b:2 c:2 d s")
t, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10 = symbols("t:11")

# Ramp code

In [6]:
def ramp(start_tempo, end_tempo, total_beats, bound_beat, bottom_bound_beat=0):
    """This integral is equal to the timecode at the bound_beat."""
    return Integral(
        1
        / (
            start_tempo / 60
            + (x / total_beats) * (S(end_tempo) / 60 - start_tempo / 60)
        ),
        (x, bottom_bound_beat, bound_beat),
    )

In [7]:
# or another way, starting with an unknown function:
_test1 = Eq(f(a, b, c, 20) + f(b, b1, c1, s), t)
display(_test1)
display(_test1.replace(f, ramp))
display(_test1.replace(f, ramp).doit())  # this is equal to tempoeq2

f(a₀, b₀, c₀, 20) + f(b₀, b₁, c₁, s) = t₀

20                         s                           
⌠                          ⌠                           
⎮          1               ⎮         1                 
⎮  ────────────────── dx + ⎮ ────────────────── dx = t₀
⎮         ⎛  a₀   b₀⎞      ⎮        ⎛  b₀   b₁⎞        
⎮       x⋅⎜- ── + ──⎟      ⎮      x⋅⎜- ── + ──⎟        
⎮  a₀     ⎝  60   60⎠      ⎮ b₀     ⎝  60   60⎠        
⎮  ── + ─────────────      ⎮ ── + ─────────────        
⎮  60         c₀           ⎮ 60         c₁             
⌡                          ⌡                           
0                          0                           

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + 20⋅a₀ - 20⋅b₀)   60⋅c₁⋅log(-b₀⋅c₁)   60
───────────────── - ───────────────────────────────── + ───────────────── - ──
     a₀ - b₀                     a₀ - b₀                     b₀ - b₁          

⋅c₁⋅log(-b₀⋅c₁ + s⋅(b₀ - b₁))     
───────────────────────────── = t₀
          b₀ - b₁                 

# Wind up+down ramp in A section

## Up Ramp

In [8]:
a, a1, b, b1, c, c1, s, t = symbols("a:2 b:2 c:2 s t")
# a = start tempo of ramp
# b = end tempo of ramp
# c = length of ramp in beats
# s = beat we are interested in
# t = timecode of that beat
tempoeq = Eq(integrate(1 / (a / 60 + (x / c) * (b / 60 - a / 60)), (x, 0, s)), t)
tempoeq

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + s⋅(a₀ - b₀))    
───────────────── - ─────────────────────────────── = t
     a₀ - b₀                    a₀ - b₀                

In [9]:
Eq(Integral(1 / (a / 60 + (x / c) * (b / 60 - a / 60)), (x, 0, s)), t)

s                          
⌠                          
⎮         1                
⎮ ────────────────── dx = t
⎮        ⎛  a₀   b₀⎞       
⎮      x⋅⎜- ── + ──⎟       
⎮ a₀     ⎝  60   60⎠       
⎮ ── + ─────────────       
⎮ 60         c₀            
⌡                          
0                          

In [10]:
s_solved = solveset(tempoeq, s).args[0]
display(s_solved)

                   ⎛60⋅c₀⋅log(-a₀⋅c₀)    ⎞
         (a₀ - b₀)⋅⎜───────────────── - t⎟
                   ⎝     a₀ - b₀         ⎠
         ─────────────────────────────────
                       60⋅c₀              
a₀⋅c₀ + ℯ                                 
──────────────────────────────────────────
                 a₀ - b₀                  

In [11]:
# @title Values for steady part
number_of_beats = 13  # @param {type:"integer"}
steady_tempo = 96  # @param {type:"integer"}

In [12]:
# @title Values for ramp part
start_tempo = 96  # @param {type:"integer"}
end_tempo = 240  # @param {type:"integer"}
length_in_beats = 20  # @param {type:"integer"}

In [13]:
temposteady = steady_tempo
beatlength = S(60) / steady_tempo
# beats = [0, S(1)/3, 2, 4]
beats = range(number_of_beats)
beattimecodes_up_ramp = [beatlength * i for i in beats]
outbeats = [
    s_solved.n(6, {a: start_tempo, b: end_tempo, c: length_in_beats, t: i}, chop=True)
    for i in beattimecodes_up_ramp
]
for i in range(len(outbeats)):
    print(beats[i], outbeats[i])

0 0
1 1.03846
2 2.15779
3 3.36430
4 4.66478
5 6.06655
6 7.57750
7 9.20612
8 10.9616
9 12.8538
10 14.8933
11 17.0917
12 19.4614


In [14]:
print(1 / 5, 1 / 4, 1 / 3, 2 / 5, 1 / 2, 3 / 5, 2 / 3, 3 / 4, 4 / 5)

0.2 0.25 0.3333333333333333 0.4 0.5 0.6 0.6666666666666666 0.75 0.8


In [15]:
display(beattimecodes_up_ramp[12].n())
display([i.n() for i in beattimecodes_up_ramp])

7.50000000000000

[0, 0.625, 1.25, 1.875, 2.5, 3.125, 3.75, 4.375, 5.0, 5.625, 6.25, 6.875, 7.5]

In [16]:
# Timecode at beat 20
solveset(tempoeq.subs({a: start_tempo, b: end_tempo, c: length_in_beats, s: 20}), t).n(
    6
)

{7.63576}

## Down Ramp

In [17]:
# b1 = end tempo of down ramp
# c1 = number of beats in down ramp
tempoeq2_not_done = Eq(
    Integral(1 / (a / 60 + (x / c) * (b / 60 - a / 60)), (x, 0, 20))
    + Integral(1 / (b / 60 + (x / c1) * (b1 / 60 - b / 60)), (x, 0, s)),
    t,
)
display(tempoeq2_not_done)
tempoeq2 = tempoeq2_not_done.doit()
display(tempoeq2)

20                         s                          
⌠                          ⌠                          
⎮          1               ⎮         1                
⎮  ────────────────── dx + ⎮ ────────────────── dx = t
⎮         ⎛  a₀   b₀⎞      ⎮        ⎛  b₀   b₁⎞       
⎮       x⋅⎜- ── + ──⎟      ⎮      x⋅⎜- ── + ──⎟       
⎮  a₀     ⎝  60   60⎠      ⎮ b₀     ⎝  60   60⎠       
⎮  ── + ─────────────      ⎮ ── + ─────────────       
⎮  60         c₀           ⎮ 60         c₁            
⌡                          ⌡                          
0                          0                          

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + 20⋅a₀ - 20⋅b₀)   60⋅c₁⋅log(-b₀⋅c₁)   60
───────────────── - ───────────────────────────────── + ───────────────── - ──
     a₀ - b₀                     a₀ - b₀                     b₀ - b₁          

⋅c₁⋅log(-b₀⋅c₁ + s⋅(b₀ - b₁))    
───────────────────────────── = t
          b₀ - b₁                

In [18]:
s2_solved = solveset(tempoeq2, s).args[0]
# display(s2_solved.args[0])
s2_solved_subbed = s2_solved.subs({a: 96, b: 240, c: 20, b1: 60, c1: 16})
display(s2_solved_subbed)
# values for slowing down ramp
beats_down_ramp = range(13, floor(16 / beatlength))
beattimecodes_down_ramp = [beatlength * i for i in beats_down_ramp]
display([i.n() for i in beattimecodes_down_ramp])
outbeats_down_ramp = [
    s2_solved_subbed.n(6, {t: i}, chop=True) for i in beattimecodes_down_ramp
]
for i in range(len(outbeats_down_ramp)):
    print(
        beats_down_ramp[i], outbeats_down_ramp[i], custom_round(outbeats_down_ramp[i])
    )  # beat values (from start of ramp)

                     -3⋅t 
                     ─────
         7/16  9/16    16 
64   80⋅2    ⋅5    ⋅ℯ     
── - ─────────────────────
3              3          

[8.125, 8.75, 9.375, 10.0, 10.625, 11.25, 11.875, 12.5, 13.125, 13.75, 14.375,
 15.0]

13 1.86990 1.8
14 4.02220 4
15 5.93649 6
16 7.63910 7.666666666666667
17 9.15343 9.2
18 10.5003 10.5
19 11.6982 11.666666666666666
20 12.7637 12.75
21 13.7113 13.75
22 14.5542 14.6
23 15.3038 15.333333333333334
24 15.9706 16


In [19]:
# total length of both ramps in seconds:
solveset(tempoeq2, t).args[0].subs({a: 96, b: 240, c: 20, b1: 60, c1: 16, s: 16}).n(6)

15.0293

# New section

In [20]:
# String accel section length in seconds
ramp(60, 180, 9 * 4, 9 * 4).doit() * 2

39.5500423920519

# Tempo relations in piece

In [21]:
from collections.abc import Iterable


def separate(item):
    if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
        return item
    else:
        return item, ""

In [27]:
tempolist_combined = (
    (t, "start"),
    t1,
    (t2, "wind accel + rit"),
    t3,
    t4,
    t5,
    t1,
    (t5, "middle"),
    t6,
)
tempolist, sections = zip(*(separate(i) for i in tempolist_combined))
tempo_equations = {
    t: t1 * 5 / 4,
    t1: 128,
    t2: t1 * 3 / 4,
    t3: t2 * 5 / 2,
    t4: t3 * 1 / 4,
    t5: t4 * 3,
    t6: t5 / 2,
}
print("equations:\n")
display(tempo_equations)
print("\nresultant tempi:", "\n------------------------------------")
for i in range(len(tempolist)):
    x = tempolist[i]
    print(x, "\t| ", N(x, 3, subs=tempo_equations), "\t| ", sections[i])

<class 'tuple'>
equations:



⎧   5⋅t₁               3⋅t₁      5⋅t₂      t₃                t₅⎫
⎨t: ────, t₁: 128, t₂: ────, t₃: ────, t₄: ──, t₅: 3⋅t₄, t₆: ──⎬
⎩    4                  4         2        4                 2 ⎭


resultant tempi: 
------------------------------------
t 	|  160. 	|  start
t1 	|  128. 	|  
t2 	|  96.0 	|  wind accel + rit
t3 	|  240. 	|  
t4 	|  60.0 	|  
t5 	|  180. 	|  
t1 	|  128. 	|  
t5 	|  180. 	|  middle
t6 	|  90.0 	|  


# Middle Section

In [23]:
# downwards ramp
middle_ramp_1_total_length = 5 + 4 + 4 + 5 + 4 + 3 + 4
middle_ramp_1_f = Eq(f(180, 90, middle_ramp_1_total_length, s, a), t)
display(middle_ramp_1_f)
middle_ramp_1 = middle_ramp_1_f.replace(f, ramp).doit()
print("length of one downwards ramp in seconds:")
middle_ramp_1_length_seconds = solveset(
    middle_ramp_1.subs({s: middle_ramp_1_total_length, a: 0}), t
).args[0]
display(middle_ramp_1_length_seconds.n(3))

print(
    "\nlength of a crotchet at 180bpm:\n\nbeat\t| length (beats)",
    "\n-----------------------------",
)
crotchet_in_seconds = S(60) / 180
crotchet_in_beats = solveset(middle_ramp_1.subs(t, crotchet_in_seconds), s).args[0] - a
beats_to_inspect = [
    0,
    middle_ramp_1_total_length * 0.25,
    middle_ramp_1_total_length * 0.5,
    middle_ramp_1_total_length * 0.75,
    middle_ramp_1_total_length - 1,
]
for location in (
    (i, crotchet_in_beats.n(3, {a: i}, chop=True)) for i in beats_to_inspect
):
    print(location[0], "\t| ", location[1])


print(
    "\n\nproportion\t| beat\t\t| time",
    "\n-----------------------------------------------",
)
for i in [0.25, 0.5, 0.75]:
    location_seconds = (i * middle_ramp_1_length_seconds).n(3)
    location = solveset(middle_ramp_1.subs({t: location_seconds, a: 0}), s).args[0].n(3)
    print(i, "\t\t| beat", location, "\t|", location_seconds, "seconds")

f(180, 90, 29, s, a₀) = t

length of one downwards ramp in seconds:


13.4


length of a crotchet at 180bpm:

beat	| length (beats) 
-----------------------------
0 	|  0.991
7.25 	|  0.868
14.5 	|  0.744
21.75 	|  0.620
28 	|  0.513


proportion	| beat		| time 
-----------------------------------------------
0.25 		| beat 9.23 	| 3.35 seconds
0.5 		| beat 17.0 	| 6.70 seconds
0.75 		| beat 23.5 	| 10.1 seconds
